# feature preprocessing 

In [1]:
import pandas as pd
import numpy as np

In [2]:
feature_df = pd.read_csv('factor.csv')
feature_df['date_time'] = pd.to_datetime(feature_df['date_time'])
feature_df['real_time'] = pd.to_datetime(feature_df['real_time'])
feature_df.set_index('date_time', inplace=True)
feature_df

,real_time,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,...,quote_asset_TVMI_14d,quote_asset_TVMO_1h,quote_asset_TVMO_4h,quote_asset_TVMO_8h,quote_asset_TVMO_1d,quote_asset_TVMO_3d,quote_asset_TVMO_7d,quote_asset_TVMO_14d,quote_asset_trade_close_ratio,quote_asset_trades_volume_ratio
date_time,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:01:00,2021-01-01 00:00:00,1609459200000,28948.19,29055.00,28706.00,29015.00,8037.588,1.609463e+12,2.321646e+08,65942.0,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3967.988170,14324.095334
2021-01-01 01:01:00,2021-01-01 01:00:00,1609462800000,29015.00,29546.42,28975.46,29448.40,19543.401,1.609466e+12,5.736394e+08,127126.0,...,45.175295,50.0,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,10352.557437,15599.447222
2021-01-01 02:01:00,2021-01-01 02:00:00,1609466400000,29443.65,29500.00,29159.16,29237.06,10049.720,1.609470e+12,2.946641e+08,69148.0,...,-28.112054,-50.0,-27.279936,-27.279936,-27.279936,-27.279936,-27.279936,-27.279936,4527.107175,13170.446948
2021-01-01 03:01:00,2021-01-01 03:00:00,1609470000000,29236.58,29379.41,29174.95,29302.11,5579.452,1.609474e+12,1.633982e+08,47082.0,...,-45.585724,-50.0,-33.438427,-33.438427,-33.438427,-33.438427,-33.438427,-33.438427,2968.095163,15587.812382
2021-01-01 04:01:00,2021-01-01 04:00:00,1609473600000,29302.11,29422.32,29086.00,29237.07,8351.683,1.609477e+12,2.443911e+08,59332.0,...,-28.206968,50.0,-23.248418,-19.327677,-19.327677,-19.327677,-19.327677,-19.327677,3666.220405,12834.484092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31 19:01:00,2024-05-31 19:00:00,1717182000000,67456.20,68059.40,67388.20,67719.90,12522.881,1.717186e+12,8.486403e+08,145735.0,...,73.920174,50.0,7.117390,-6.911857,24.556714,27.372176,23.349364,9.083566,6955.204950,37611.615387
2024-05-31 20:01:00,2024-05-31 20:00:00,1717185600000,67720.00,67834.40,67633.10,67755.00,3592.345,1.717189e+12,2.433091e+08,57772.0,...,-57.452807,-50.0,-38.314375,-45.870267,-19.278904,-17.968322,-10.373184,-7.060205,1700.663998,32076.119960
2024-05-31 21:01:00,2024-05-31 21:00:00,1717189200000,67755.00,67763.00,67557.70,67602.30,2543.573,1.717193e+12,1.720847e+08,42362.0,...,-72.810497,-50.0,-32.398619,-40.685989,-24.477476,-23.234839,-14.308584,-8.949065,1089.447706,28955.005668


In [3]:
def handle_return(df, column_names):
    def handle_missing_values(column):
        if column.isnull().any():
            if column.skew() > 0.5 or column.skew() < -0.5:
                fill_value = column.median()
            else:
                fill_value = column.mean()
            column.fillna(fill_value, inplace=True)
        return column
    for col in column_names:
        if df[col].dtype in ['float64', 'int64']: 
            df[col] = handle_missing_values(df[col])
    return df
column_return = feature_df.columns[12:19].tolist()
feature_df = handle_return(feature_df, column_return)

In [4]:
feature_df1 = feature_df.copy()

def handle_data1(df, column_names):
    def remove_outliers(column):
        D_M = column.median()
        D_M1 = (column - D_M).abs().median()
        upper_limit = D_M + 5 * D_M1
        lower_limit = D_M - 5 * D_M1
        return np.clip(column, lower_limit, upper_limit)

    def handle_missing_values(column):
        if column.isnull().any():
            if column.skew() > 0.5 or column.skew() < -0.5:
                fill_value = column.median()
            else:
                fill_value = column.mean()
            column.fillna(fill_value, inplace=True)
        return column

    def standardize_data(column):
        mean = column.mean()
        std_dev = column.std()
        return (column - mean) / std_dev

    for col in column_names:
        if df[col].dtype in ['float64', 'int64']: 
            df[col] = remove_outliers(df[col])
            df[col] = handle_missing_values(df[col])
            df[col] = standardize_data(df[col])

    return df

column_names = feature_df1.columns[8:11].tolist() + feature_df1.columns[19:].tolist()
feature_df1 = handle_data1(feature_df1, column_names)
feature_df1

,real_time,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,...,quote_asset_TVMI_14d,quote_asset_TVMO_1h,quote_asset_TVMO_4h,quote_asset_TVMO_8h,quote_asset_TVMO_1d,quote_asset_TVMO_3d,quote_asset_TVMO_7d,quote_asset_TVMO_14d,quote_asset_trade_close_ratio,quote_asset_trades_volume_ratio
date_time,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:01:00,2021-01-01 00:00:00,1609459200000,28948.19,29055.00,28706.00,29015.00,8037.588,1.609463e+12,-0.810893,-0.870633,...,0.149086,0.000000,0.000000,-0.292480,-0.292864,-0.282457,-0.275407,-0.267356,-0.678223,-0.603115
2021-01-01 01:01:00,2021-01-01 01:00:00,1609462800000,29015.00,29546.42,28975.46,29448.40,19543.401,1.609466e+12,0.055891,-0.209986,...,0.908805,1.020621,1.255676,1.418920,2.240157,2.265562,2.268556,2.300471,0.464003,-0.428163
2021-01-01 02:01:00,2021-01-01 02:00:00,1609466400000,29443.65,29500.00,29159.16,29237.06,10049.720,1.609470e+12,-0.652247,-0.836016,...,-0.323678,-0.801269,-0.623758,-0.776798,-1.165470,-1.730686,-2.378062,-2.835184,-0.578194,-0.761372
2021-01-01 03:01:00,2021-01-01 03:00:00,1609470000000,29236.58,29379.41,29174.95,29302.11,5579.452,1.609474e+12,-0.985446,-1.074278,...,-0.617536,-0.801585,-0.773531,-0.951777,-1.446226,-2.143822,-2.819371,-2.835184,-0.857108,-0.429760
2021-01-01 04:01:00,2021-01-01 04:00:00,1609473600000,29302.11,29422.32,29086.00,29237.07,8351.683,1.609477e+12,-0.779858,-0.942006,...,-0.325275,1.020621,-0.525712,-0.550855,-0.802938,-1.197216,-1.642325,-2.026949,-0.732211,-0.807459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-31 19:01:00,2024-05-31 19:00:00,1717182000000,67456.20,68059.40,67388.20,67719.90,12522.881,1.717186e+12,0.753940,-0.009051,...,1.392212,1.020621,0.212779,-0.198090,1.197687,1.935596,2.268556,1.173085,-0.143798,2.591449
2024-05-31 20:01:00,2024-05-31 20:00:00,1717185600000,67720.00,67834.40,67633.10,67755.00,3592.345,1.717189e+12,-0.782604,-0.958851,...,-0.817106,-0.808124,-0.892113,-1.304997,-0.800714,-1.106025,-0.813861,-0.645231,-1.083857,1.832094
2024-05-31 21:01:00,2024-05-31 21:00:00,1717189200000,67755.00,67763.00,67557.70,67602.30,2543.573,1.717193e+12,-0.963397,-1.125244,...,-1.075378,-0.859060,-0.748243,-1.157698,-1.037710,-1.459324,-1.177962,-0.857978,-1.193206,1.403942


In [5]:
print(feature_df1.isna().sum())

real_time                          0
timestamp                          0
open                               0
high                               0
low                                0
                                  ..
quote_asset_TVMO_3d                0
quote_asset_TVMO_7d                0
quote_asset_TVMO_14d               0
quote_asset_trade_close_ratio      0
quote_asset_trades_volume_ratio    0
Length: 227, dtype: int64


In [ ]:
feature_df2 = feature_df.copy()

def handle_data2(df, column_names):
    def remove_outliers(column):
        D_M = column.median()
        D_M1 = (column - D_M).abs().median()
        upper_limit = D_M + 5 * D_M1
        lower_limit = D_M - 5 * D_M1
        return np.clip(column, lower_limit, upper_limit)

    def handle_missing_values(column):
        if column.isnull().any():
            if column.skew() > 0.5 or column.skew() < -0.5:
                fill_value = column.median()
            else:
                fill_value = column.mean()
            column.fillna(fill_value, inplace=True)
        return column

    def normalize_data(column):
        X_min = column.min()
        X_max = column.max()
        return (column - X_min) / (X_max - X_min) if (X_max - X_min) != 0 else 0

    for col in column_names:
        if df[col].dtype in ['float64', 'int64']: 
            df[col] = remove_outliers(df[col])
            df[col] = handle_missing_values(df[col])
            df[col] = normalize_data(df[col])

    return df

column_names = feature_df2.columns[8:11].tolist() + feature_df2.columns[19:].tolist()
feature_df2 = handle_data2(feature_df2, column_names)
feature_df2

In [ ]:
print(feature_df2.isna().sum())

In [ ]:
feature_df1.to_csv('feature_df1.csv')
feature_df2.to_csv('feature_df2.csv')